In [2]:
import os
import torch
import torchvision

import h5py
import deepdish as dd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

import matplotlib.pyplot as plt

from PIL import Image

import time
import pickle as pkl

In /usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /usr/remote/anaconda-3.7-2020-05-28/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /usr/

In [12]:
n_epochs = 2
batch_size_train = 4# was 64
batch_size_test = 4
learning_rate = 0.001
momentum = 0.1
log_interval = 31
tt_factor = 0.8
orientation = 'AP'
num_imgs = 2500

random_seed = 1
#torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [13]:
''' 
    NOTES: 
    - it's "No Finding" not "No findings"
    - it's "Pleural_Thickening" not "Pleural_thickening"
    - it's not "Nodule Mass", but rather "Nodule" and "Mass" separately
'''
disease_map = {"Atelectasis" : 0, "Consolidation" : 1, "Infiltration" : 2, "Pneumothorax": 3, "Edema": 4,
               "Emphysema": 5, "Fibrosis": 6, "Effusion" : 7, "Pneumonia" : 8, "Pleural_Thickening" : 9,
               "Cardiomegaly" : 10, "Nodule" : 11, "Mass" : 12, "Hernia" : 13, "No Finding" : 14 }


In [14]:
class GetLoader(torch.utils.data.Dataset):
    '''
        Params: data - the data dictionary
                view - the orientation you want to look at
                diseases - the diseases you would like to look at
                num_imgs - the number of images of each disease you would like
                factor - the ratio of training and testing data
                typ - 0 for training, 1 for testing
    '''
    def __init__(self, data, view, diseases, num_imgs, factor, typ, transforms=None):
        
        #private data
        self.root = os.path.join('data/sorted_images',)
        self.data = data # dict object
        self.transforms = transforms
        self.len_data = 0
        datalist = []
        
        #Creating the datalist
        for i in range(len(diseases)):
            if len(data[view][diseases[i]]) <= num_imgs: #if the folder has less images than the desired number of images
                if typ == 0:
                    start = 0
                    end = int(len(data[view][diseases[i]])*factor)
                else:
                    start = int(len(data[view][diseases[i]])*factor)
                    end = -1
            else:
                if typ == 0:
                    start = 0
                    end = int(num_imgs*factor)
                else:
                    start = int(num_imgs*factor)
                    end = num_imgs
            datalist.append(self.data[view][diseases[i]][start:end])
        
        for item in datalist:
            self.len_data += len(item)
        
        
        self.img_paths = []
        self.img_labels = []
        
        for dis in datalist:
            for data in dis:
                #creating the image path
                data['img_path'] = os.path.join(self.root, data['classes'][0], view, data['img_name'])            
                diseases = data['classes']
                
                # TODO: we want the 1-hot vector for full-class net, not for 1 class net
                # what is below is very much hardcoded
#                 if data['classes'][0] != "No Finding":
#                     label = 1
#                 else:
#                     label = 0
                
                # label = torch.Tensor([label])

                one_hot = [0] * 15
            
                for c in data['classes']:
                    hot_index = disease_map[c]
                    one_hot[hot_index] = 1
                    
                label = torch.Tensor(one_hot)
                
                #modifying the attributes
                self.img_paths.append(data['img_path'])
                self.img_labels.append(label)
                # self.img_labels.append(data['label_tensor'])
            

    def __getitem__(self, item):
        
        img_path, img_label = self.img_paths[item], self.img_labels[item]
  
        # TODO: fix this hot fix -> recreate data object with underscore in name
        img_path = img_path.replace('No Finding', 'No_Finding')
        try:
            img = Image.open(img_path).convert('RGB')
        except:
            img_path = img_path.replace('/AP/', '/PA/')
            img = Image.open(img_path).convert('RGB')
        self.cur_img_path = img_path

        if self.transforms is not None:
            for t in self.transforms:
                img = t(img)         

        return img, img_label
    
    def get_img_path(self):
        return self.cur_img_path

    def __len__(self):
        return self.len_data

### load in datasets

In [15]:
pkl_load = open('dataset.pickle', 'rb')
data = pkl.load(pkl_load)
pkl_load.close()
print("data loaded in successfully")

data loaded in successfully


In [16]:
print(data['AP'].keys())

dict_keys(['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'No Finding'])


In [17]:
transforms = [torchvision.transforms.ToTensor()] # no longer grayscale conversion
#0 = train, 1 = test
dataset_train = GetLoader(data, orientation, ['Atelectasis', 'Consolidation', 'Infiltration', 
                                              'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 
                                              'Effusion', 'Pneumonia', 'Pleural_Thickening', 
                                              'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'No Finding'],
                          num_imgs, tt_factor, 0, transforms)
dataset_test = GetLoader(data, orientation, ['Atelectasis', 'Consolidation', 'Infiltration', 
                                              'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 
                                              'Effusion', 'Pneumonia', 'Pleural_Thickening', 
                                              'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'No Finding'],
                          num_imgs, tt_factor, 1, transforms)
print(len(dataset_train))
print(len(dataset_test))

20078
5014


In [18]:
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size_train, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size_train, shuffle=True, num_workers=1)

print(len(train_loader))
print(len(test_loader))

5020
1254


In [19]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11', pretrained=True)

Using cache found in /home/npp002/.cache/torch/hub/pytorch_vision_v0.6.0


In [20]:
model.classifier[6] = nn.Sequential(*[
    nn.Linear(in_features=4096, out_features=500, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=500, out_features=15, bias=True),
    nn.Sigmoid()
])

In [21]:
layers = [0, 3, 6, 8, 11, 13, 16, 18]
outs_per_layer = [64, 128, 256, 256, 512, 512, 512, 512]
for i, out_f in zip(layers, outs_per_layer):
    model.features[i] = nn.Sequential(*[model.features[i], nn.BatchNorm2d(out_f)])

In [22]:
model.children

<bound method Module.children of VGG(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (7): ReLU(inplace=True)
    (8): Sequential(
      (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1,

In [23]:
network = model
#network = torchvision.models.vgg11(pretrained=False)
#network.fc = nn.Linear(512, 15)
network = torch.nn.DataParallel(network)
network.cuda()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)
print('Trainable params: {}'.format(sum(p.numel() for p in network.parameters() if p.requires_grad)))

Trainable params: 130827855


In [24]:
## source: https://gist.github.com/the-bass/cae9f3976866776dea17a5049013258d
def hard_binary_accuracy(batch, labels):
    batch = torch.round(batch)
    confusion_matrix = batch / labels
    
    """ Returns the confusion matrix for the values in the `prediction` and `truth`
    tensors, i.e. the amount of positions where the values of `prediction`
    and `truth` are
    - 1 and 1 (True Positive)
    - 1 and 0 (False Positive)
    - 0 and 0 (True Negative)
    - 0 and 1 (False Negative)
    """

    confusion_matrix = batch / labels
    # Element-wise division of the 2 tensors returns a new tensor which holds a
    # unique value for each case:
    #   1     where prediction and truth are 1 (True Positive)
    #   inf   where prediction is 1 and truth is 0 (False Positive)
    #   nan   where prediction and truth are 0 (True Negative)
    #   0     where prediction is 0 and truth is 1 (False Negative)

    true_positives = torch.sum(confusion_matrix == 1).item()
    false_positives = torch.sum(confusion_matrix == float('inf')).item()
    true_negatives = torch.sum(torch.isnan(confusion_matrix)).item()
    false_negatives = torch.sum(confusion_matrix == 0).item()

    return true_positives, false_positives, true_negatives, false_negatives


In [25]:
b = torch.Tensor([[[1, 1], [0, 0]], 
                  [[1, 1], [0, 0]]])

l = torch.Tensor([[[1, 1], [0, 0]], 
                  [[1, 1], [0, 0]]])
hard_binary_accuracy(b, l)

(4, 0, 4, 0)

In [26]:
def max_pred_conversion(batch, label):
    pass

In [27]:
def train(epoch):

    network.train()
    train_losses = []
    num_tested = 0
    net_loss = 0
    
    total_true_pos, total_false_pos, total_true_neg, total_false_neg = 0, 0, 0, 0
    
    for batch_idx, (data, target) in enumerate(train_loader):  # (output - actual )
        
        if torch.cuda.is_available:
            data, target = data.cuda(), target.cuda()

        output = network(data)
        
        criterion = torch.nn.BCELoss()  # 1 0 1 0 0 0 
        loss = criterion(output, target)
        loss.backward()
        
        net_loss += loss.detach().cpu().item()
        train_losses.append(loss.item())

        num_tested += len(data)
        
        #if True or batch_idx == len(train_loader) - 1:
        optimizer.step()
        optimizer.zero_grad()

        pred = output.detach().clone().cpu()
        target = target.cpu()

        true_pos, false_pos, true_neg, false_neg = hard_binary_accuracy(pred, target)
        total_true_pos += true_pos
        total_false_pos += false_pos
        total_true_neg += true_neg
        total_false_neg += false_neg
            
        
    net_loss /= len(train_loader)
    return net_loss, total_true_pos, total_false_pos, total_true_neg, total_false_neg


In [28]:
def test():
    
    network.eval()
    test_losses = []
    net_loss = 0    

    total_true_pos, total_false_pos, total_true_neg, total_false_neg = 0, 0, 0, 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            if torch.cuda.is_available:
                data, target = data.cuda(), target.cuda()
            
            output = network(data)
            criterion = torch.nn.BCELoss()
            net_loss += criterion(output, target).item()

            pred = output.detach().clone().cpu()
            target = target.cpu()
            
            true_pos, false_pos, true_neg, false_neg = hard_binary_accuracy(pred, target)
            total_true_pos += true_pos
            total_false_pos += false_pos
            total_true_neg += true_neg
            total_false_neg += false_neg
            
        
    net_loss /= len(train_loader)
    return net_loss, total_true_pos, total_false_pos, total_true_neg, total_false_neg


In [29]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

1
GeForce RTX 2080 Ti
True


In [30]:
train_sample_size = len(train_loader) * batch_size_train
test_sample_size = len(test_loader) * batch_size_test
print(train_sample_size, test_sample_size)

20080 5016


In [31]:
print("Starting Training...")
begin_time = time.time()

train_losses = []
test_losses = []

train_acc = []
test_acc = []

train_true_pos = []
train_false_pos = []
train_true_neg = []
train_false_neg = []

test_true_pos = []
test_false_pos = []
test_true_neg = []
test_false_neg = []



# patience_limit = 25
# patience = 0
# min_test_loss = 10

for epoch in range(1, n_epochs + 1):
    start_time = time.time()

    loss_train, true_pos_train, false_pos_train, true_neg_train, false_neg_train = train(epoch)
    loss_test, true_pos_test, false_pos_test, true_neg_test, false_neg_test = test()
    
#     print("tp / fp / tn / fn : ", true_pos_test, false_pos_test, true_neg_test, false_neg_test )
    train_losses.append(loss_train)
    test_losses.append(loss_test)
    
    n_correct_train = true_pos_train + true_neg_train
    n_correct_test = true_pos_test + true_neg_test
    
    acc_train = 100. * n_correct_train / (train_sample_size * 15)
    acc_test = 100. * n_correct_test / (test_sample_size * 15)
    
    train_acc.append(acc_train)
    test_acc.append(acc_test)
    
    true_pos_rate_train = 100. * true_pos_train / (train_sample_size * 15)
    true_neg_rate_train = 100. * true_neg_train / (train_sample_size * 15)
    false_pos_rate_train = 100. * false_pos_train / (train_sample_size * 15)
    false_neg_rate_train = 100. * false_neg_train / (train_sample_size * 15)
    
    true_pos_rate_test = 100. * true_pos_test / (test_sample_size * 15)
    true_neg_rate_test = 100. * true_neg_test / (test_sample_size * 15)
    false_pos_rate_test = 100. * false_pos_test / (test_sample_size * 15)
    false_neg_rate_test = 100. * false_neg_test / (test_sample_size * 15)

    train_true_pos.append(true_pos_train)
    train_false_pos.append(false_pos_train)
    train_true_neg.append(true_neg_train)
    train_false_neg.append(false_neg_train)

    test_true_pos.append(true_pos_test)
    test_false_pos.append(false_pos_test)
    test_true_neg.append(true_neg_test)
    test_false_neg.append(false_neg_test)
        
    elapsed_time = time.time() - start_time
    
    print("-----------Epoch " + str(epoch) + " (time =", round(elapsed_time, 2), "s  aka  ", round(elapsed_time / 60, 2), " mins) ----------------")
    print('Train set: Avg. loss: {:.4f},  Accuracy: {}/{} ({:.2f}%)'.format(
        loss_train, n_correct_train, train_sample_size * 15, acc_train))
    print('''
          True Positive: {:.2f}%
          True Negative: {:.2f}%
          False Positive: {:.2f}%
          False Negative {:.2f}%
          '''.format(true_pos_rate_train, true_neg_rate_train, false_pos_rate_train, false_neg_rate_train))    
    print('\nTest set:  Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        loss_test, n_correct_test, test_sample_size * 15, acc_test))
    print('''
          True Positive: {:.2f}%
          True Negative: {:.2f}%
          False Positive: {:.2f}%
          False Negative {:.2f}%
          '''.format(true_pos_rate_test, true_neg_rate_test, false_pos_rate_test, false_neg_rate_test))
    
end_time = time.time() - begin_time

print("Training complete (time =", round(end_time, 2), "s)")
    

Starting Training...
-----------Epoch 1 (time = 1837.74 s  aka   30.63  mins) ----------------
Train set: Avg. loss: 0.3918,  Accuracy: 257921/301200 (85.63%)

          True Positive: 0.41%
          True Negative: 85.22%
          False Positive: 1.65%
          False Negative 12.71%
          

Test set:  Avg. loss: 0.0901, Accuracy: 64925/75240 (86.29%)

          True Positive: 0.00%
          True Negative: 86.29%
          False Positive: 0.00%
          False Negative 13.67%
          
-----------Epoch 2 (time = 1837.96 s  aka   30.63  mins) ----------------
Train set: Avg. loss: 0.3609,  Accuracy: 261535/301200 (86.83%)

          True Positive: 0.03%
          True Negative: 86.80%
          False Positive: 0.07%
          False Negative 13.08%
          

Test set:  Avg. loss: 0.0892, Accuracy: 64925/75240 (86.29%)

          True Positive: 0.00%
          True Negative: 86.29%
          False Positive: 0.00%
          False Negative 13.67%
          
Training complete (time

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.title("Loss per Epoch")
plt.plot(train_losses, "-b",  label="train")
plt.plot(test_losses, "-r", label="test")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(loc="upper right")

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.title("Accuracy per Epoch")
plt.plot(train_acc, "-b", label="train")
plt.plot(test_acc, "-r", label="test")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(loc="upper right")

In [ ]:
# torch.save(network.state_dict(), 
#            os.path.join('/home/nick/workspace/saved_models/classifier_nodule.pth'))